In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_rerun_links.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 2)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 13000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:14000]:
    try:
        print(url)
        if driver_count >=20:
            driver_count = 0
            print("need to close driver")
            driver.quit()
            driver = get_driver()
            print("new driver initialize****************")
        driver.get(url)
        driver = wait_to_page_load(driver)
        try:
            agent_name = driver.find_element_by_class_name("AgentContent__name").text
        except:
            agent_name = ""
        try:
            time.sleep(0.5)
            profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
        except:
            profile_url = ""
        try:
            tag = driver.find_element_by_class_name("pill").text
        except:
            tag = ""
        try:
            role = driver.find_element_by_class_name("AgentContent__team").text
        except:
            role = ""
        try:
            location = driver.find_element_by_class_name("AgentContent__location").text
        except:
            location = ""
        contact_dict = collect_agent_contact_details(driver)
        agent_details = get_agent_details(driver)
        social_media_dict,social_media_links_str = social_media_details(driver)
        other_info = collect_other_info(driver)
        data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
        data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
        with open("kw_data-emailSelenium_13k_14k.csv",'a',newline='',encoding='utf-8') as f:
            data_df.to_csv(f, mode='a',header=f.tell()==0)
        print(url_count,url)
        url_count+=1
        driver_count+=1
        driver.delete_all_cookies()
        print("***********************************")
    except:
        pass
driver.quit()


https://kw.com/agent/UPA-6846178354870034432-9
Timeout happened no page load
13000 https://kw.com/agent/UPA-6846178354870034432-9
***********************************
https://kw.com/agent/UPA-6587385305396985859-5
Page load happened
13001 https://kw.com/agent/UPA-6587385305396985859-5
***********************************
https://kw.com/agent/UPA-6587385350568902657-9
Page load happened
13002 https://kw.com/agent/UPA-6587385350568902657-9
***********************************
https://kw.com/agent/UPA-6587385184027463683-1
Page load happened
13003 https://kw.com/agent/UPA-6587385184027463683-1
***********************************
https://kw.com/agent/UPA-6587385445944008710-9
Page load happened
13004 https://kw.com/agent/UPA-6587385445944008710-9
***********************************
https://kw.com/agent/UPA-6587385186722009092-3
Page load happened
13005 https://kw.com/agent/UPA-6587385186722009092-3
***********************************
https://kw.com/agent/UPA-6714241010340466688-8
Page load ha

13051 https://kw.com/agent/UPA-6587385267859230723-5
***********************************
https://kw.com/agent/UPA-6587385297310314501-2
Page load happened
13052 https://kw.com/agent/UPA-6587385297310314501-2
***********************************
https://kw.com/agent/UPA-6587385318503047173-0
Page load happened
13053 https://kw.com/agent/UPA-6587385318503047173-0
***********************************
https://kw.com/agent/UPA-6760325821335703552-8
Page load happened
13054 https://kw.com/agent/UPA-6760325821335703552-8
***********************************
https://kw.com/agent/UPA-6587385318737928198-6
Page load happened
13055 https://kw.com/agent/UPA-6587385318737928198-6
***********************************
https://kw.com/agent/UPA-6587385208011743232-8
Page load happened
13056 https://kw.com/agent/UPA-6587385208011743232-8
***********************************
https://kw.com/agent/UPA-6707295187036454912-0
Page load happened
13057 https://kw.com/agent/UPA-6707295187036454912-0
*****************

Page load happened
13103 https://kw.com/agent/UPA-6773279596932767744-6
***********************************
https://kw.com/agent/UPA-6587384988791312384-5
Page load happened
13104 https://kw.com/agent/UPA-6587384988791312384-5
***********************************
https://kw.com/agent/UPA-6695154290524262400-6
Page load happened
13105 https://kw.com/agent/UPA-6695154290524262400-6
***********************************
https://kw.com/agent/UPA-6592997756490711040-1
Page load happened
13106 https://kw.com/agent/UPA-6592997756490711040-1
***********************************
https://kw.com/agent/UPA-6587385245282881539-3
Page load happened
13107 https://kw.com/agent/UPA-6587385245282881539-3
***********************************
https://kw.com/agent/UPA-6724779929245036544-7
Page load happened
13108 https://kw.com/agent/UPA-6724779929245036544-7
***********************************
https://kw.com/agent/UPA-6587385197130092547-5
Page load happened
13109 https://kw.com/agent/UPA-6587385197130092547-

Page load happened
13155 https://kw.com/agent/UPA-6587385098366930950-3
***********************************
https://kw.com/agent/UPA-6587385368203972608-1
Page load happened
13156 https://kw.com/agent/UPA-6587385368203972608-1
***********************************
https://kw.com/agent/UPA-6793620086610669568-2
Page load happened
13157 https://kw.com/agent/UPA-6793620086610669568-2
***********************************
https://kw.com/agent/UPA-6587385306963742721-4
Page load happened
13158 https://kw.com/agent/UPA-6587385306963742721-4
***********************************
https://kw.com/agent/UPA-6587385082722750469-0
Page load happened
13159 https://kw.com/agent/UPA-6587385082722750469-0
***********************************
https://kw.com/agent/UPA-6645350757178707973-1
need to close driver
new driver initialize****************
Timeout happened no page load
13160 https://kw.com/agent/UPA-6645350757178707973-1
***********************************
https://kw.com/agent/UPA-6587385148737839108-5


Page load happened
13207 https://kw.com/agent/UPA-6587385148804947969-5
***********************************
https://kw.com/agent/UPA-6645350799755010048-5
Page load happened
13208 https://kw.com/agent/UPA-6645350799755010048-5
***********************************
https://kw.com/agent/UPA-6833451749558521856-5
Page load happened
13209 https://kw.com/agent/UPA-6833451749558521856-5
***********************************
https://kw.com/agent/UPA-6740392826528292864-4
Page load happened
13210 https://kw.com/agent/UPA-6740392826528292864-4
***********************************
https://kw.com/agent/UPA-6587385166434869252-5
Timeout happened no page load
13211 https://kw.com/agent/UPA-6587385166434869252-5
***********************************
https://kw.com/agent/UPA-6587385180146089988-6
Page load happened
13212 https://kw.com/agent/UPA-6587385180146089988-6
***********************************
https://kw.com/agent/UPA-6587385300969164809-6
Page load happened
13213 https://kw.com/agent/UPA-658738530

...

Page load happened
13770 https://kw.com/agent/UPA-6800534113395851264-2
***********************************
https://kw.com/agent/UPA-6587385292616204294-9
Page load happened
13771 https://kw.com/agent/UPA-6587385292616204294-9
***********************************
https://kw.com/agent/UPA-6587385152001605638-0
Timeout happened no page load
13772 https://kw.com/agent/UPA-6587385152001605638-0
***********************************
https://kw.com/agent/UPA-6640292643991207938-2
Page load happened
13773 https://kw.com/agent/UPA-6640292643991207938-2
***********************************
https://kw.com/agent/UPA-6587385436182634496-8
Page load happened
13774 https://kw.com/agent/UPA-6587385436182634496-8
***********************************
https://kw.com/agent/UPA-6587385443396083719-4
Page load happened
13775 https://kw.com/agent/UPA-6587385443396083719-4
***********************************
https://kw.com/agent/UPA-6587385203965722627-9
Page load happened
13776 https://kw.com/agent/UPA-658738520

Page load happened
13821 https://kw.com/agent/UPA-6587385410598932484-5
***********************************
https://kw.com/agent/UPA-6587385429529755648-6
Page load happened
13822 https://kw.com/agent/UPA-6587385429529755648-6
***********************************
https://kw.com/agent/UPA-6734158128051212288-3
Page load happened
13823 https://kw.com/agent/UPA-6734158128051212288-3
***********************************
https://kw.com/agent/UPA-6587385232516419584-2
Page load happened
13824 https://kw.com/agent/UPA-6587385232516419584-2
***********************************
https://kw.com/agent/UPA-6806316819668553728-0
Page load happened
13825 https://kw.com/agent/UPA-6806316819668553728-0
***********************************
https://kw.com/agent/UPA-6780255427094355968-0
Page load happened
13826 https://kw.com/agent/UPA-6780255427094355968-0
***********************************
https://kw.com/agent/UPA-6587385323246825473-0
Page load happened
13827 https://kw.com/agent/UPA-6587385323246825473-

Page load happened
13873 https://kw.com/agent/UPA-6634130634391818244-6
***********************************
https://kw.com/agent/UPA-6587385162364350465-2
Page load happened
13874 https://kw.com/agent/UPA-6587385162364350465-2
***********************************
https://kw.com/agent/UPA-6593183108613939204-8
Page load happened
13875 https://kw.com/agent/UPA-6593183108613939204-8
***********************************
https://kw.com/agent/UPA-6841529444257021952-9
Page load happened
13876 https://kw.com/agent/UPA-6841529444257021952-9
***********************************
https://kw.com/agent/UPA-6587384997696737286-2
Page load happened
13877 https://kw.com/agent/UPA-6587384997696737286-2
***********************************
https://kw.com/agent/UPA-6739532523233980416-3
Page load happened
13878 https://kw.com/agent/UPA-6739532523233980416-3
***********************************
https://kw.com/agent/UPA-6587385227232133125-9
Page load happened
13879 https://kw.com/agent/UPA-6587385227232133125-

Page load happened
13925 https://kw.com/agent/UPA-6587385229294624769-2
***********************************
https://kw.com/agent/UPA-6790041182807388160-9
Page load happened
13926 https://kw.com/agent/UPA-6790041182807388160-9
***********************************
https://kw.com/agent/UPA-6608309913731436544-7
Timeout happened no page load
13927 https://kw.com/agent/UPA-6608309913731436544-7
***********************************
https://kw.com/agent/UPA-6587384989839577090-3
Page load happened
13928 https://kw.com/agent/UPA-6587384989839577090-3
***********************************
https://kw.com/agent/UPA-6785677338089824256-9
Page load happened
13929 https://kw.com/agent/UPA-6785677338089824256-9
***********************************
https://kw.com/agent/UPA-6587385257156993026-0
Page load happened
13930 https://kw.com/agent/UPA-6587385257156993026-0
***********************************
https://kw.com/agent/UPA-6592949300770545664-9
Page load happened
13931 https://kw.com/agent/UPA-659294930

Page load happened
13977 https://kw.com/agent/UPA-6645349998793682947-1
***********************************
https://kw.com/agent/UPA-6587385195891531778-6
Page load happened
13978 https://kw.com/agent/UPA-6587385195891531778-6
***********************************
https://kw.com/agent/UPA-6701240829325520896-6
Page load happened
13979 https://kw.com/agent/UPA-6701240829325520896-6
***********************************
https://kw.com/agent/UPA-6587385259115896836-1
need to close driver
new driver initialize****************
Page load happened
13980 https://kw.com/agent/UPA-6587385259115896836-1
***********************************
https://kw.com/agent/UPA-6587385412047663111-0
Page load happened
13981 https://kw.com/agent/UPA-6587385412047663111-0
***********************************
https://kw.com/agent/UPA-6824073951410634752-3
Page load happened
13982 https://kw.com/agent/UPA-6824073951410634752-3
***********************************
https://kw.com/agent/UPA-6838161743222140928-5
Page load h

In [4]:
driver.quit()